# Text Generation
mkultra is set up for convenient text generation with transformers generation and sampling tools.

You can load a SoftPrompt object and simply concatenate it into your context string, where it shows up as a human-readable tag that tokenizes to the underlying number of tokens.

In [ ]:
#@title Setup for Colab only
#!pip install transformers
#!pip install git+git://github.com/corolla-johnson/mkultra.git#egg=mkultra

In [1]:
from transformers.pipelines import pipeline
from mkultra.models.inference import GPT2SoftPromptLM
from mkultra.tokenizers import GPT2SPTokenizerFast
from mkultra.soft_prompt import SoftPrompt

In [2]:
# You'll need to instantiate mkultra's model and tokenizer classes.
model = GPT2SoftPromptLM.from_pretrained("gpt2")
tokenizer = GPT2SPTokenizerFast.from_pretrained("gpt2")
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [3]:
# SoftPrompts may be loaded in one of several ways.
# sp = SoftPrompt.from_file("soft_prompts/neuromancer.pt")
# sp = SoftPrompt.from_inputs_embeds(inputs_embeds)
# sp = SoftPrompt.from_tuning_model(model)
# We will instantiate an SP from a string for testing. This should behave identically to the text.
sp = SoftPrompt.from_string("With the court firmly balkanized into three distinct factions, Princess Charlotte had her work cut out for her.",
                            model=model, tokenizer=tokenizer)

In [4]:
# Information about an sp can be printed with
print(sp)

FromString (2021-06-08 20:47:03.855460)
Length: 22
UUID:   453df955-c0a4-497c-bc58-3049250ff059
Description:
Created from string 'With the court firmly balkanized into three distinct factions, Princess Charlotte had her work cut out for her.'


In [5]:
# Use 'get_tag_str()' to get a tag string that you can add to your context.
prompt = sp.get_tag_str() + " She"

# The addition operator also works:
# prompt = sp + " She"

In [6]:
# The tag string conveniently contains the SP's name, part of its GUID,
# and a series of '@'s which represent individual soft tokens.
print(prompt)

<FromString-453df955-@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@> She


In [7]:
# These tokens help you budget your context.
prompt_len = len(tokenizer.encode(prompt))
print(f"Length of soft prompt: {len(tokenizer.encode(sp.get_tag_str()))}")
print(f"Length of full prompt: {prompt_len}")

Length of soft prompt: 22
Length of full prompt: 23


In [8]:
# Generation is as usual.
output = generator( prompt,
                    do_sample=True,
                    min_length=prompt_len+100,
                    max_length=prompt_len+100,
                    repetition_penalty=1.7,
                    top_p=0.8,
                    temperature=0.7,
                    use_cache=True,
                    return_full_text=True)

print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
[{'generated_text': "<FromString-453df955-@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@><@> She could only rely on the aid of her sister, who was still young and inexperienced.\nThe court had to decide if she should go with their leader or stay behind in order for him to find a suitable partner within this new world as well? The two women were very close friends; they even dated when Charlotte wanted them together more than once! They couldn't afford such an awkward relationship now that it seemed so much easier since there wasn' no way he'd be able help out… yet somehow both"}]


In [ ]:
# It is also fine to use generate() instead of a pipeline
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

output = model.generate(input_ids)

print(output)

In [ ]:
# If you decide to write your own sampler instead of a pipeline,
# be aware that the special token logits will be very high.
# This is accounted for when using model.generate() or pipelines,
# but you will need to exclude special tokens when sampling.

output = model(input_ids)